# imports and loading of input data

In [ ]:
import os
import pandas as pd
import numpy as np
import xarray as xr
import datetime
import matplotlib.pyplot as plt
import pickle
import itertools

In [ ]:
import sys
# setting path
sys.path.append('../')

from ruins.processing import drain_cap

In [ ]:
# load primary data
data = xr.load_dataset('../data/hydro_krummh.nc').to_dataframe()
data.head()

In [ ]:
#load cached events
with open('cache/events.pkl', 'rb') as file:
    events = pickle.load(file)
with open('cache/canals.pkl', 'rb') as file:
    canal_par = pickle.load(file)
with open('cache/pump.pkl', 'rb') as file:
    pumpcap_fit = pickle.load(file)
events

# Generate dataframe with all events to be simulated

In [ ]:
for RCP in ['rcp26','rcp45','rcp85', 'ref']:
    print(RCP)
    for ClimateTime in ['2065']:
        if RCP == 'rcp26':
            if ClimateTime == '2010':
                SLRs = [35,122,221]  # SLR in 2030 SSP126, outer range of low/medium confidence
                prec_increases = [0.9, 1, 1.1]
            if ClimateTime == '2065':
                SLRs = [154, 432, 848] # SLR in 2090 SSP126, outer range of low/medium confidence
                prec_increases = [0.8, 0.9, 1, 1.1, 1.2]
            comb26 = list(itertools.product(
                            ['rcp26'],
                              SLRs,
                              prec_increases,
                              events,
                              [str(r) for r in canal_par],
                              [4,6],
                              [0,50,200],
                              [1,4500,6000],
                              data.filter(regex='Prec').columns,
                              [np.nan],
                              [np.nan]
                             )
                        )
            
        if RCP == 'ref':
            if ClimateTime == '2010':
                SLRs = [0]  # SLR in 2030 SSP126, outer range of low/medium confidence
                prec_increases = [0.9, 1, 1.1]
            if ClimateTime == '2065':
                SLRs = [0] # SLR in 2090 SSP126, outer range of low/medium confidence
                prec_increases = [0.8, 0.9, 1, 1.1, 1.2]
            combref = list(itertools.product(
                            ['ref'],
                              SLRs,
                              prec_increases,
                              events,
                              [str(r) for r in canal_par],
                              [4,6],
                              [0,50,200],
                              [1,4500,6000],
                              data.filter(regex='Prec').columns,
                              [np.nan],
                              [np.nan]
                             )
                            )

        if RCP == 'rcp45':
            if ClimateTime == '2010':
                SLRs = [29,126,232]  # SLR in 2030 SSP245
                prec_increases = [0.9, 1, 1.1]
            if ClimateTime == '2065':
                SLRs = [249, 522, 918] # SLR in 2090 SSP245
                prec_increases = [0.8, 0.9, 1, 1.1, 1.2]

                comb45 = list(itertools.product(
                            ['rcp45'],
                              SLRs,
                              prec_increases,
                              events,
                              [str(r) for r in canal_par],
                              [4,6],
                              [0,50,200],
                              [1,4500,6000],
                              data.filter(regex='Prec').columns,
                              [np.nan],
                              [np.nan]
                             )
                        )
        if RCP == 'rcp85':
            if ClimateTime == '2010':
                SLRs = [34,125,231]  # SLR in 2030 SSP126, outer range of low/medium confidence
                prec_increases = [0.9, 1, 1.1]
            if ClimateTime == '2065':
                SLRs = [379, 730, 1143, 1676] # SLR in 2090 SSP126, lower range of low/medium confidence, upper range: 1143 for medium confidence,1676 for low confidence 
                prec_increases = [0.8, 0.9, 1, 1.1, 1.2, 1.3]
        
        
            comb85 = list(itertools.product(
                            ['rcp85'],
                              SLRs,
                              prec_increases,
                              events,
                              [str(r) for r in canal_par],
                              [4,6],
                              [0,50,200],
                              [1,4500,6000],
                              data.filter(regex='Prec').columns
                            )
                         )

#print(len(comb26)+len(comb45)+len(comb85)+len(combref))
#comb26[0:2]

comb26.extend(comb45)
comb26.extend(comb85)
comb26.extend(combref)
print(len(comb26))

combis = pd.DataFrame(comb26, columns= ['rcp','SLR','prec_incr',
                                        'event','canal_para', 'canal_area',
                                        'forecast_pump', 'maxdh','raindissagg',
                                        'max_store','energy_cons'])

In [ ]:
combis

# do simulations - parallel processing would be nice

In [ ]:
i = 0

for index, row in combis.iterrows():
    if (i%5000 == 0):
        print(i)
        print(datetime.datetime.now())
    t1 = events[row.event]
    t2 = events[row.event]+datetime.timedelta(days=14)
    hourly_recharge = data[row.raindissagg][t1:t2]
    days = (t2-t1).days
    hours = days*24 + 1
    tide = data['wl_Knock_Outer'][t1:t2] + row.SLR     # add SLR to tide water level [mm]
    hourly_recharge = hourly_recharge.rolling("12h").mean() # smoothen rainfall signal to account for water redistribution within catchment
    hourly_recharge *= row.prec_incr  # scale hourly recharge by change in climate forcing
    wig = tide*0     # no wind induced gradient -> this may be used to simulate a local flooding in a subcatchment
    x = pd.DataFrame.from_dict(
        {
            'recharge' : hourly_recharge.fillna(0),
            'h_tide' : tide.fillna(100),
            'wig' : wig.fillna(0),
        }
    )
    canal_para= tuple(float(v.lstrip("('").rstrip("')")) for v in row.canal_para.split(", "))
    x['h_store'], q_pump, x['h_min'], x['flow_rec'], pump_cost, x['store'] = drain_cap.storage_model(
        forcing_data = x,
        canal_par = canal_para,
        v_store = 0,
        h_store_target = -1350,
        canal_area = row.canal_area,
        h_forecast_pump = row.forecast_pump,
        h_grad_pump_max = row.maxdh,
        pump_par = pumpcap_fit)
    combis.loc[index,'max_store'] = max(x['store'])
    combis.loc[index,'energy_cons'] = sum(pump_cost)
    i +=1

In [ ]:
#combis.to_csv("res_final.csv")
#results = pd.read_csv("res_final.csv", index=0)